## <a href='https://spacy.io/universe/project/spacy-transformers' style='text-decoration:none'>Spacy-transformers</a>
- <a href='https://github.com/explosion/spacy-transformers' style='text-decoration:none'>Github</a>
- blog: <a href='https://explosion.ai/blog/spacy-transformers' style='text-decoration:none'>spaCy meets Transformers: Fine-tune BERT, XLNet and GPT-2</a>

In [ ]:
#!pip install --upgrade spacy
#!pip install spacy-transformers==0.5.2   # otherwise cause issue 'init_kwargs', https://github.com/explosion/spacy-transformers/issues/139

In [1]:
import sys
sys.path.insert(-1, '/xdisk/msurdeanu/fanluo/miniconda3/lib/python3.7/site-packages') 
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal

In [2]:
#!pip install spacy-transformers              # spacy-transformers-0.6.2 
# !python -m spacy download en_trf_robertabase_lg 

import en_trf_robertabase_lg
nlp = en_trf_robertabase_lg.load()            # load pretrained models  

In [5]:
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    torch.set_default_tensor_type("torch.cuda.FloatTensor")

In [3]:
doc = nlp("Here is some text to encode.")
assert doc.tensor.shape == (7, 768)           # Always has one row per token

In [7]:
doc._.trf_word_pieces_                        # String values of the wordpieces

['<s>', 'Here', 'Ġis', 'Ġsome', 'Ġtext', 'Ġto', 'Ġencode', '.', '</s>']

In [8]:
doc._.trf_word_pieces                         # Wordpiece IDs (note: *not* spaCy's hash values!)

[0, 11773, 16, 103, 2788, 7, 46855, 4, 2]

In [9]:
doc._.trf_alignment                           # Alignment between spaCy tokens and wordpieces

[[1], [2], [3], [4], [5], [6], [7]]

In [10]:
# one output row per wordpiece
assert len(doc._.trf_last_hidden_state) == len(doc._.trf_word_pieces)

In [11]:
assert_almost_equal(doc.tensor.sum(axis=0), doc._.trf_last_hidden_state.sum(axis=0), decimal=5)

In [ ]:
span = doc[2:4] 
assert numpy.array_equal(span.tensor, doc.tensor[2:4])

In [ ]:
apple1 = nlp("Apple shares rose on the news.")
apple2 = nlp("Apple sold fewer iPhones this quarter.")
apple3 = nlp("Apple pie is delicious.")
print(apple1[0].similarity(apple2[0]))  # 0.73428553
print(apple1[0].similarity(apple3[0]))  # 0.43365782

In [ ]:
#create a model locally using the examples/init_model.py script.